# Monte Carlo Risk Analysis

This notebook demonstrates Monte Carlo simulation for portfolio risk analysis and retirement planning.

## Overview

Monte Carlo simulation generates thousands of potential future scenarios by:
- Sampling from historical return distributions
- Incorporating volatility and correlation patterns
- Modeling various withdrawal strategies
- Calculating probability of success/failure

## Use Cases

1. **Retirement Planning**: Assess sustainability of withdrawal rates
2. **Portfolio Construction**: Estimate range of possible outcomes
3. **Risk Assessment**: Calculate Value at Risk (VaR) and Conditional VaR
4. **Goal Planning**: Probability of reaching financial goals

In [ ]:
# Setup
import warnings

warnings.filterwarnings("ignore")

# Set environment
import os

import numpy as np
import plotly.graph_objects as go

os.environ["DYNACONF_ENV"] = "development"

from config import logger

logger.info("Notebook initialized")

## 1. Load Historical Data

Load historical returns to calibrate the simulation.

In [ ]:
from finbot.utils.data_collection_utils.yfinance.get_history import get_history

# Load portfolio assets
portfolio_assets = ["SPY", "TLT"]  # 60/40 stock/bond portfolio
asset_data = {}

for asset in portfolio_assets:
    try:
        asset_data[asset] = get_history(asset, adjust_price=True)
        print(f"✓ Loaded {asset}: {len(asset_data[asset])} days")
    except Exception as e:
        print(f"✗ Error loading {asset}: {e}")

# Calculate daily returns
returns = {}
for asset, data in asset_data.items():
    returns[asset] = data["Close"].pct_change().dropna()
    print(f"\n{asset} Statistics:")
    print(f"  Mean Daily Return: {returns[asset].mean():.4%}")
    print(f"  Daily Volatility: {returns[asset].std():.4%}")
    print(f"  Annualized Return (252 days): {returns[asset].mean() * 252:.2%}")
    print(f"  Annualized Volatility: {returns[asset].std() * np.sqrt(252):.2%}")

## 2. Run Monte Carlo Simulation

Simulate 10,000 possible future scenarios for a 60/40 portfolio.

In [ ]:
from finbot.services.simulation.monte_carlo.monte_carlo_simulator import MonteCarloSimulator

# Portfolio parameters
initial_value = 1_000_000  # $1M starting portfolio
allocation_spy = 0.60  # 60% stocks
allocation_tlt = 0.40  # 40% bonds
num_simulations = 10_000
num_years = 30
trading_days_per_year = 252

# Initialize simulator
simulator = MonteCarloSimulator(
    returns_data=returns, allocations={"SPY": allocation_spy, "TLT": allocation_tlt}, initial_value=initial_value
)

# Run simulation
print(f"Running {num_simulations:,} simulations for {num_years} years...")
simulation_results = simulator.run(
    num_simulations=num_simulations,
    num_periods=num_years * trading_days_per_year,
    seed=42,  # For reproducibility
)

print(f"✓ Simulation complete: {simulation_results.shape}")
print(f"  Simulations: {simulation_results.shape[0]:,}")
print(f"  Time periods: {simulation_results.shape[1]:,}")

## 3. Visualize Simulation Results

Plot the fan chart showing range of possible outcomes.

In [ ]:
from finbot.services.simulation.monte_carlo.visualization import plot_trials

# Convert to annual data points for cleaner visualization
annual_indices = np.arange(0, simulation_results.shape[1], trading_days_per_year)
annual_results = simulation_results[:, annual_indices]

# Create fan chart
fig = plot_trials(
    trials=annual_results, percentiles=[10, 25, 50, 75, 90], title="Monte Carlo Simulation: 60/40 Portfolio (30 Years)"
)

fig.show()

## 4. Calculate Key Risk Metrics

Compute percentiles, Value at Risk (VaR), and probability of loss.

In [ ]:
# Calculate final values (after 30 years)
final_values = simulation_results[:, -1]

# Percentiles
percentiles = [5, 10, 25, 50, 75, 90, 95]
percentile_values = np.percentile(final_values, percentiles)

print("Final Portfolio Value Percentiles (30 years):")
print("=" * 60)
for pct, val in zip(percentiles, percentile_values, strict=False):
    print(f"  {pct:2d}th percentile: ${val:>12,.0f}")

# Value at Risk (VaR)
var_95 = np.percentile(final_values, 5)
var_99 = np.percentile(final_values, 1)

print("\nValue at Risk (VaR):")
print("=" * 60)
print(f"  95% VaR (5th percentile): ${var_95:,.0f}")
print(f"  99% VaR (1st percentile): ${var_99:,.0f}")

# Probability of loss
prob_loss = (final_values < initial_value).mean() * 100
prob_double = (final_values >= initial_value * 2).mean() * 100
prob_triple = (final_values >= initial_value * 3).mean() * 100

print("\nProbability of Outcomes:")
print("=" * 60)
print(f"  Loss (ending < ${initial_value:,.0f}): {prob_loss:.1f}%")
print(f"  Doubling (ending >= ${initial_value * 2:,.0f}): {prob_double:.1f}%")
print(f"  Tripling (ending >= ${initial_value * 3:,.0f}): {prob_triple:.1f}%")

# Expected value and median
expected_value = final_values.mean()
median_value = np.median(final_values)

print("\nCentral Tendency:")
print("=" * 60)
print(f"  Expected Value (mean): ${expected_value:,.0f}")
print(f"  Median Value: ${median_value:,.0f}")
print(f"  Expected Multiple: {expected_value / initial_value:.2f}x")

## 5. Distribution Histogram

Visualize the distribution of final portfolio values.

In [ ]:
from finbot.services.simulation.monte_carlo.visualization import plot_histogram

# Create histogram of final values
fig = plot_histogram(values=final_values, title="Distribution of Final Portfolio Values (30 Years)", bins=100)

# Add vertical lines for key percentiles
fig.add_vline(x=var_95, line_dash="dash", line_color="red", annotation_text="95% VaR")
fig.add_vline(x=median_value, line_dash="dash", line_color="green", annotation_text="Median")

fig.show()

## 6. Retirement Withdrawal Analysis

Test the sustainability of different withdrawal rates (4% rule, 3.5% rule, etc.).

In [ ]:
def test_withdrawal_rate(simulation_results, initial_value, withdrawal_rate, inflation=0.03):
    """
    Test if portfolio survives with given withdrawal rate.

    Args:
        simulation_results: Array of simulated portfolio values
        initial_value: Starting portfolio value
        withdrawal_rate: Annual withdrawal rate (e.g., 0.04 for 4%)
        inflation: Annual inflation rate for withdrawal adjustments

    Returns:
        Success rate (percentage of simulations that don't run out of money)
    """
    annual_withdrawal = initial_value * withdrawal_rate
    daily_withdrawal = annual_withdrawal / 252

    # Adjust for inflation annually
    success_count = 0

    for sim in simulation_results:
        portfolio = initial_value
        current_withdrawal = daily_withdrawal

        for day in range(1, len(sim)):
            # Apply return
            daily_return = (sim[day] - sim[day - 1]) / sim[day - 1]
            portfolio = portfolio * (1 + daily_return)

            # Withdraw
            portfolio -= current_withdrawal

            # Adjust for inflation annually
            if day % 252 == 0:
                current_withdrawal *= 1 + inflation

            # Check if ran out of money
            if portfolio <= 0:
                break

        if portfolio > 0:
            success_count += 1

    return (success_count / len(simulation_results)) * 100


# Test different withdrawal rates
withdrawal_rates = [0.03, 0.035, 0.04, 0.045, 0.05]
success_rates = []

print("Withdrawal Rate Sustainability (30 years, 3% inflation):")
print("=" * 60)

for rate in withdrawal_rates:
    success_rate = test_withdrawal_rate(
        simulation_results=simulation_results, initial_value=initial_value, withdrawal_rate=rate, inflation=0.03
    )
    success_rates.append(success_rate)
    print(f"  {rate:.1%} withdrawal: {success_rate:.1f}% success rate")

# Plot success rates
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=[r * 100 for r in withdrawal_rates],
        y=success_rates,
        mode="lines+markers",
        line=dict(color="blue", width=3),
        marker=dict(size=10),
    )
)

# Add reference line at 95% success
fig.add_hline(y=95, line_dash="dash", line_color="green", annotation_text="95% Target")

fig.update_layout(
    title="Withdrawal Rate vs Success Rate (60/40 Portfolio, 30 Years)",
    xaxis_title="Withdrawal Rate (%)",
    yaxis_title="Success Rate (%)",
    height=500,
)

fig.show()

## Key Findings

From Monte Carlo analysis of a 60/40 portfolio:

1. **Wide Range of Outcomes**: Even with the same strategy, final values can vary by 10x or more due to sequence of returns risk

2. **Value at Risk**: The 5th percentile (95% VaR) shows the worst-case scenario with 95% confidence - critical for conservative planning

3. **Asymmetric Returns**: Median outcome is typically lower than mean outcome due to positive skew in equity returns

4. **Withdrawal Rate Sensitivity**: Small changes in withdrawal rate (3.5% vs 4.0%) can significantly impact success probability

5. **4% Rule**: Classic 4% withdrawal rule may have ~85-95% success rate for 60/40 portfolio over 30 years (varies by period)

6. **Sequence Risk**: Early market downturns combined with withdrawals can deplete portfolio even if long-term returns are positive

7. **Planning Conservatively**: Aiming for 95%+ success rate suggests lower withdrawal rates (3-3.5%) may be more prudent

## Next Steps

- Test different asset allocations (80/20, 40/60, all-stock)
- Model dynamic withdrawal strategies (adjust based on portfolio performance)
- Incorporate Social Security or pension income
- Test different rebalancing frequencies
- Model tax-efficient withdrawal strategies
- Compare historical vs parametric (normal distribution) simulations